In [28]:
import random
import util
from test_framework.key import generate_key_pair, generate_bip340_key_pair, ECKey, ECPubKey, jacobi_symbol
from test_framework.key import SECP256K1_FIELD_SIZE, SECP256K1_ORDER
from test_framework.messages import sha256
from test_framework.script import tagged_hash

k, R = generate_key_pair()
y = R.get_y()
minus_y = SECP256K1_FIELD_SIZE - y
print("y = {}".format(y))
print("-y = {}".format(minus_y))
print("y is {}".format("odd" if y % 2 else "even"))
print("-y is {}".format("odd" if minus_y % 2 else "even"))
print("k is {} a valid nonce".format("" if y % 2 == 0 else "not"))
print("-k is {} a valid nonce".format("" if minus_y % 2 == 0 else "not"))

y = 13559482138227799626734527953556176018836073146593135805526466394426745362928
-y = 102232607099088395796836457055131731834433911519047428233931117613482089308735
y is even
-y is odd
k is  a valid nonce
-k is not a valid nonce


In [3]:
k, R = generate_key_pair()
R_x = R.get_x()
R_y = R.get_y()
print("R_x: {}".format(R_x))
print("R_y: {}".format(R_y))
minus_k = SECP256K1_ORDER - k.secret
minus_k_key, minus_R = generate_key_pair(minus_k)
minus_R_x = minus_R.get_x()
minus_R_y = minus_R.get_y()
print("minus_R_x: {}".format(minus_R_x))
print("minus_R_y: {}\n".format(minus_R_y))
assert R_x == minus_R_x
assert SECP256K1_FIELD_SIZE - R_y == minus_R_y

R_x: 53089450365856660873746794545791032443351319662115159088690721692461984342167
R_y: 91970744997821119035534967173148238180413515560579319663143539655773497091007
minus_R_x: 53089450365856660873746794545791032443351319662115159088690721692461984342167
minus_R_y: 23821344239495076388036017835539669672856469105061244376314044352135337580656



In [29]:
msg = sha256(b'message')
d, P = generate_key_pair()
if P.get_y() % 2 != 0:
    d.negate()
    P.negate()

k, R = generate_key_pair()
if R.get_y() % 2 != 0:
    k.negate()

R_x_bytes = R.get_bytes()
P_bytes = P.get_bytes()
h_bytes = tagged_hash("BIP0340/challenge", R_x_bytes + P_bytes + msg)
h = ECKey().set(h_bytes)
s = k + h * d
print("R: {}".format(R))
print("s: {}\n".format(s.get_bytes().hex()))
sig = R_x_bytes + s.get_bytes()
print("Signature: {}\n".format(sig.hex()))
assert P.verify_schnorr(sig, msg)

R: 99287886f290d62d2924cdf13e99c60d22c0fb357f94dae723e576f3e3f7ae30
s: ddd3cdaa9bedb097b34c34772e4325ae56c88dccc9c95c4018666348ad372a67

Signature: 99287886f290d62d2924cdf13e99c60d22c0fb357f94dae723e576f3e3f7ae30ddd3cdaa9bedb097b34c34772e4325ae56c88dccc9c95c4018666348ad372a67



In [37]:
msg = sha256(b'message')
aux = sha256(b'random auxiliary data')
d, P = generate_bip340_key_pair()
print("message = {}".format(msg.hex()))
print("pubkey={}\n".format(P.get_bytes().hex()))
t = (d.secret ^ int.from_bytes(tagged_hash("BIP0340/aux", aux), 'big')).to_bytes(32, 'big')
rand = tagged_hash("BIP0340/nonce", t + P.get_bytes() + msg)
k, R = generate_key_pair(rand)
if R.get_y() % 2 != 0:
    k.negate()
print("nonce: {}".format(k))
print("nonce point: {}\n".format(R))
R_x_bytes = R.get_bytes()
P_bytes = P.get_bytes()
h_bytes = tagged_hash("BIP0340/challenge", R_x_bytes + P_bytes + msg)
h = ECKey().set(h_bytes)
s = k + h * d
print("R: {}".format(R))
print("s: {}\n".format(s.get_bytes().hex()))
sig = R_x_bytes + s.get_bytes()
print("Signature: {}\n".format(sig.hex()))
sig2 = d.sign_schnorr(msg, aux)
assert P.verify_schnorr(sig, msg)
assert P.verify_schnorr(sig2, msg)

message = ab530a13e45914982b79f9b7e3fba994cfd1f3fb22f71cea1afbf02b460c6d1d
pubkey=bcb8efa5789baa1755d3f599e6db024e7ba3f3808c09bdd95a04624cc1425f3e

nonce: 43083883860340328596388228062243319847994337646097710974223116911382313045790
nonce point: 365191a89d35ae6d55ffcffcca1d4b65e2565386b5b162ecf4d541ddcffabfa7

R: 365191a89d35ae6d55ffcffcca1d4b65e2565386b5b162ecf4d541ddcffabfa7
s: fc5cce0b14ac2fe84d2cdf2b98316f82cb890e310a65a057a75f5344406df808

Signature: 365191a89d35ae6d55ffcffcca1d4b65e2565386b5b162ecf4d541ddcffabfa7fc5cce0b14ac2fe84d2cdf2b98316f82cb890e310a65a057a75f5344406df808

